In [ ]:
#checking we are working on the right virtual environment

In [ ]:
import sys
sys.executable
#!pip3 freeze > requirements.txt
#!python --version

# IMPORT PACKAGES

In [ ]:
import numpy as np
import pandas as pd

from ydata_profiling import ProfileReport

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from scipy.stats import spearmanr

from sklearn.model_selection import StratifiedShuffleSplit
from math import log2

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PowerTransformer
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import datetime

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from yellowbrick.regressor import PredictionError, ResidualsPlot

from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import PolynomialFeatures

from joblib import dump, load

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor

let's define a few funcion that will help us later

In [ ]:
def visualizer():
    viz = PredictionError(model, bestfit=False, alpha=0.2) #it gives problems with bestfit=True
    viz.fit(X_train, y_train)
    viz.score(X_test, y_test)
    scatter_kwargs = {'color': (222/255, 151/255, 228/255)}
    viz.show(bgcolor='k', scatter_kwargs=scatter_kwargs)
    #with ResidualsPlot sometimes kernel dies
    #viz = ResidualsPlot(model, test_alpha=0.1) #, hist=False, qqplot=True
    #viz.fit(X_train, y_train)
    #viz.score(X_test, y_test)
    #viz.show()

In [ ]:
def metrics():
    print('R2:')
    print('train', r2_score(y_train, model.predict(X_train)))
    print('test:', r2_score(y_test, model.predict(X_test)))
    print('\n')
    print('RMSE')
    print('train', np.sqrt(mean_squared_error(y_train, model.predict(X_train))))
    print('test:', np.sqrt(mean_squared_error(y_test, model.predict(X_test))))
    print('\n')
    print('MAPE:')
    print('train', mean_absolute_percentage_error(y_train, model.predict(X_train))) #Note here that the output is not a percentage in the range [0, 100] and a value of 100 does not mean 100% but 1e2.
    print('test:', mean_absolute_percentage_error(y_test, model.predict(X_test))) #Note here that the output is not a percentage in the range [0, 100] and a value of 100 does not mean 100% but 1e2.
    print('\n')

# IMPORT DATASET

In [ ]:
df = pd.read_csv('./Project datasets-20230327/flight_dataset.csv', index_col=0)
df.head()

resetting index, just to be sure

In [ ]:
df.reset_index(inplace=True, drop=True)

# EDA

In [ ]:
#profile = ProfileReport(df, title="FLIGHTS - Pandas Profiling Report", explorative=True)
#profile.to_file("flights.html")

checking for nans

In [ ]:
df.shape

In [ ]:
df.isna().sum()

checking for duplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
########
########

## PRICE

looking at price distribution

In [ ]:
fig, axs = plt.subplots(figsize=(12,10))
sns.histplot(data=df, x='price', kde=True, color='purple')
plt.title('PRICE HISTOGRAM')

price seems to be pretty skewed

we'll probably have to work on it later

In [ ]:
fig, axs = plt.subplots(figsize=(12,10))
sns.boxplot(data=df, x='price', color='violet')
plt.title('PRICE BOXPLOT')

In [ ]:
#########
########

### CITIES

price boxplots on source and destionation cities

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12,10))

g1 = sns.boxplot(data=df, x='price', y='source_city', ax=axs[0])
g1.set(title = 'BOXPLOT PRICE - SOURCE CITY')

g2 = sns.boxplot(data=df, x='price', y='destination_city', ax=axs[1])
g2.set(title = 'BOXPLOT PRICE - DESTINATION CITY')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12,10))

g1 = sns.histplot(data=df, x='price', hue='source_city', kde=True, ax=axs[0])
g1.set(title = 'COUNTPLOT & KDES PRICE - SOURCE CITY')

g2 = sns.histplot(data=df, x='price', hue='destination_city', kde=True, ax=axs[1])
g2.set(title = 'COUNTPLOT & KDES PRICE - DESTINATION CITY')

Indipendently from source or destination city, more economy flights are sold than business flights

In [ ]:
city_cross = pd.crosstab(df['source_city'], df['destination_city'])
city_cross

In [ ]:
for i in city_cross.columns:
    city_cross[f'{i}'].plot(kind='barh')
    #plt.grid()
    plt.title(f'HOW MANY FLIGHTS ARRIVE AT {i}')
    plt.show()

In [ ]:
########
########

### STOPS

price boxplots on the number of stops made

In [ ]:
plt.subplots(figsize=(12,5))
sns.boxplot(data=df, x='price', y='stops')
plt.title('BOXPLOT PRICE - STOPS')

interestingly zero stops flights tend to be the more economic ones, this may be due to the fact that they make smaller voyages


two or more stops have a lot of outliers, may that be last minute flights?

In [ ]:
plt.subplots(figsize=(12,5))
sns.histplot(data=df, x='price', hue='stops', kde=True)
plt.title('COUNTPLOT & KDES PRICE - STOPS')

two or more stops flights are the least sold.

one stop flights sell a lot more than the other two categories and sell for both economy and business trips.

zero stops flights sell mostly economy flights

In [ ]:
stops_cross = pd.crosstab(df['destination_city'], df['stops'])
stops_cross

In [ ]:
for i in stops_cross.columns:
    stops_cross[f'{i}'].plot(kind='barh')
    #plt.grid()
    plt.title(f'HOW MANY FLIGHTS {i} STOP')
    plt.show()

Dheli doen't work a lot with two or more stops flights, but works a lot more than the others with zero stop fligths

In [ ]:
#########
#########

### ARRIVAL TIME

price boxplots on the arrival time

In [ ]:
plt.subplots(figsize=(12,5))
sns.boxplot(data=df, x='price', y='arrival_time')
plt.title('BOXPLOT PRICE - ARRIVAL TIME')

as expected the most economic flights are the one that arrive at destination in the late night or early in the morining

also afternoon flights is reasonable that are a little cheaper than the reaming arrival time

In [ ]:
plt.subplots(figsize=(12,5))
sns.histplot(data=df, x='price', hue='arrival_time', kde=True)
plt.title('COUNTPLOT & KDES PRICE - ARRIVAL TIME')

most of tickets is sold at a low price. Tickets indeferently from the arrival_time, except for early_morning flights, seems to have a little peak at 40.000-60.000 price. That's probably due to business flights sales, while the first peak is for economy sales

In [ ]:
arr_cross = pd.crosstab(df['destination_city'], df['arrival_time'])
arr_cross

In [ ]:
for i in arr_cross.columns:
    arr_cross[f'{i}'].plot(kind='barh')
    #plt.grid()
    plt.title(f'HOW MANY FLIGHTS IN THE {i} ')
    plt.show()

it's interesting to notice that Dheli seems to work a lot in the Late_Night, but pretty much nothing in the Early_Morning

In [ ]:
########
########

### CLASS

price boxplots on class

In [ ]:
plt.subplots(figsize=(12,5))
sns.boxplot(data=df, x='price', y='class')
plt.title('BOXPLOT PRICE - CLASS')

well.. as exepcted economy flights are cheaper

plotting countplot and kdes for economy price and business price

In [ ]:
plt.subplots(figsize=(12,5))
sns.histplot(data=df, x='price', hue='class', kde=True)
plt.title('COUNTPLOT & KDES PRICE - STOPS')

business pricess seems to be pretty much Normally distriubuted, but we can't say the same for economy prices

In [ ]:
########
########

### DURATION

price - duration scatterplot

In [ ]:
sns.scatterplot(data=df, x='price', y='duration')
plt.title('SCATTERPLOT PRICE - DURATION')

paradoxaly longer duration flights are not the more expensive

In [ ]:
#########
#########

## CORRELATION

correlation matrix

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")

linear correlation is really low, so we expect that linear models won't perform that good.

we still try to predict with a linear model, and then we will compare it's error metric with the one of the other models

Spearman correlation coefficient for categorical variables

In [ ]:
df_cat = df.copy()
for i in df_cat.columns:
    if df_cat[i].dtypes == 'int64' or df_cat[i].dtypes == 'float64':       
           df_cat = df_cat.drop(i, axis=1)
            
for i in df_cat.columns:
    df_cat[i] = df_cat[i].astype('category')

print('Spearman correlation coefficient: \n')
for col1, col2 in combinations(df_cat.columns, 2):
    correlation, p_value = spearmanr(df_cat[col1].cat.codes, df_cat[col2].cat.codes)
    print(col1, '-', col2, ': ', correlation)

In [ ]:
#############
#############

# PRE-PROCESSING

we've already seen that there are non missing values and no duplicates, so we will look for outliers

## OUTLIER DETECTION WITH BOXPLOTS

In [ ]:
fig, axs = plt.subplots(figsize=(12,10))
sns.boxplot(data=df, x='price', color='violet')
plt.title('PRICE BOXPLOT')

calculating interquantile range and conisdering outlier each observation that is over the treshold of +/-1.5*IQR

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

out = df[(df['price'] < Q1 - 1.5*IQR) | (df['price'] > Q3 + 1.5*IQR)]
print(len(out), 'outliers were found')

removing outliers from the original df and definig a new one

In [ ]:
not_out = df.loc[~df.index.isin(out.index)]
not_out.head()

In [ ]:
#subsets make sense
not_out.shape[0]+out.shape[0] == df.shape[0]

In [ ]:
fig, axs = plt.subplots(figsize=(12,10))
sns.boxplot(data=not_out, x='price', color='violet')
plt.title('PRICE BOXPLOT - NO OUTLIERS')

we subscribe the original df with our new df without outliers

In [ ]:
df = not_out.copy()

In [ ]:
##########
##########

## STRATIFIED TRAIN-TEST SPLIT

In [ ]:
#code was inspired by asking to chatgpt:
#"do you how can i make a stratified train test split for a regression problem?"

### LOOKING FOR THE RIGHT NUMBER OF BINS

now we look for the right number of bins to make our stratified train-test split

In [ ]:
#aggiungere titoli ai grafici

In [ ]:
#square-root rule
sns.histplot(data=df, x='price', bins=round(np.sqrt(df.shape[0])))

In [ ]:
#Sturges' rule

sns.histplot(data=df, x='price', bins=round(log2(df.shape[0])))

In [ ]:
#Freedman-Diaconis' rule

#we removed some observatioms, so we have to calculate again the IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

sns.histplot(data=df, x='price', bins=round((2*IQR)/(df.shape[0]**(1/3))))

In [ ]:
#Scott's rule
sns.histplot(data=df, x='price', bins=round((3.5*np.std(df['price']))/(df.shape[0]**(1/3))))

Sturges' rule seems to make not too many bins and not too few bins

### ACTUALLY TRAIN-TEST SPLITTING

In [ ]:
# create bins for the target variable
bins = pd.qcut(df['price'], q=round(log2(df.shape[0])), labels=False)

# create a stratified train-test split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2) #, random_state=42
for train_index, test_index in split.split(df, bins):
    train = df.iloc[train_index]
    test = df.iloc[test_index]

In [ ]:
X_train, y_train = train.drop("price", axis=1), train["price"].values.flatten()
X_test, y_test = test.drop("price", axis=1), test["price"].values.flatten()

In [ ]:
#no observation was lost while splitiing train and test
X_train.shape[0]+X_test.shape[0] == df.shape[0]

In [ ]:
#########
########

## COLUMN TRANSFORMER

In [ ]:
# otherwiese the power transform won't accept the array
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
power = PowerTransformer(method='box-cox')
y_train = power.fit_transform(y_train)
y_test = power.transform(y_test)

plotting the transformed target variable

In [ ]:
ytrbc = pd.DataFrame(y_train, columns=['ytrain_boxcox'])

fig, axs = plt.subplots(figsize=(12,10), dpi=300)
sns.histplot(data=ytrbc, x='ytrain_boxcox', kde=True, color=(222/255, 151/255, 228/255))
axs.set_facecolor((0,0,0))
plt.title('PRICE DENSITY DISTRIBUTION - TRANSFORMED', color='white')
plt.xlabel('Price', color='white')
plt.ylabel('Count', color='white')
plt.tick_params(colors='white')
plt.grid(alpha=0.2)
fig.patch.set_facecolor((0, 0, 0))
plt.show()

In [ ]:
ct = ColumnTransformer([
    ('hot', OneHotEncoder(drop='first'), ['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city', 'class']), 
    ('ord', OrdinalEncoder(categories=[['zero', 'one', 'two_or_more']]), ['stops']),
    ('pow', PowerTransformer(method='box-cox'), ['duration']),
    ('std', StandardScaler(), ['duration']),
    ('drop', 'drop', ['flight'])
], remainder='passthrough') #passthrough is not useful this time, I'm just used to use it and I think it's good practice
ct
#30 features in output

In [ ]:
loo_pipe = Pipeline([
    ('loo', LeaveOneOutEncoder()),
    ('std', StandardScaler())
])

ctl = ColumnTransformer([
    ('loo', loo_pipe, ['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city', 'class']), 
    ('ord', OrdinalEncoder(categories=[['zero', 'one', 'two_or_more']]), ['stops']),
    ('pow', PowerTransformer(method='box-cox'), ['duration']),
    ('std', StandardScaler(), ['duration']),
    ('drop', 'drop', ['flight'])
], remainder='passthrough') #passthrough is not useful this time, I'm just used to use it and I think it's good practice
ctl
#10 features in output

In [ ]:
########
########

# MODEL 1 - LINEAR MODEL

## LINEAR REGRESSION

In [ ]:
#from sklearn.feature_selection import RFECV

In [ ]:
model = LinearRegression()
#rfecv = RFECV(estimator=model, cv=3)
kbest = SelectKBest(score_func=mutual_info_regression)
poly = PolynomialFeatures()
pipe = Pipeline([
    ('column_transformer', ct),
    #('selection', rfecv),
    ('selection', kbest),
    ('polynomial', poly),
    ('regressor', model)
])
pipe

In [ ]:
param_grid = {
    #'selection__min_features_to_select': [30, 20, 10],
    'selection__k': [30, 20, 10],
    'polynomial__degree': [1, 2, 3],
    'regressor__fit_intercept': [True]
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring
)

model_grid = grid.fit(X_train, y_train.ravel())
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
dump(model, 'flights_linear_poly.pkl')
dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')

In [ ]:
########

lets try with leave one out instead of one hot encoder

In [ ]:
model = LinearRegression()
#rfecv = RFECV(estimator=model, cv=3)
kbest = SelectKBest(score_func=mutual_info_regression)
poly = PolynomialFeatures()
pipel = Pipeline([
    ('column_transformer', ctl),
    #('selection', rfecv),
    ('selection', kbest),
    ('polynomial', poly),
    ('regressor', model)
])
pipel

In [ ]:
param_grid = {
    #'selection__min_features_to_select': [10, 8, 5],
    'selection__k': [10, 8, 5, 3, 1],
    'polynomial__degree': [1, 2, 3],
    'regressor__fit_intercept': [True]
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV(
    estimator=pipel,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    #n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring
)

model_grid = grid.fit(X_train, y_train)
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')

In [ ]:
#########
#########

## SVR
for the SVR, but also for the tree based methods, we will just use the one-hot pipe, not also the loo pipe

SVR is too slow on a df of this scale, so we don't even bother to run it

In [ ]:
model = SVR()
kbest = SelectKBest(score_func=mutual_info_regression)
poly = PolynomialFeatures()
pipe = Pipeline([
    ('column_transformer', ct),
    #('selection', kbest),
    #('polynomial', poly),
    ('regressor', model)
])
pipe

In [ ]:
#valutare di fare la random grid search, che 2025 non sono pochissimissimi
param_grid = {
    #'selection__k': [30, 20, 10],
    #'polynomial__degree': [1, 2, 3],
    'regressor__kernel': ['linear', 'rbf', 'sigmoid'],
    'regressor__gamma': ['scale', 'auto', 0.1, 1, 10],
    'regressor__C': [0.1, 1, 10, 100, 1000]#,
    #'regressor__epsilon': [0.01, 0.1, 1]
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    #n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring
)

model_grid = grid.fit(X_train, y_train.ravel())
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')

In [ ]:
##########
##########

## SGD REGRESSOR

In [ ]:
model = SGDRegressor()
#kbest = SelectKBest(score_func=mutual_info_regression)
poly = PolynomialFeatures()
pipe = Pipeline([
    ('column_transformer', ct),
    #('selection', kbest),
    ('polynomial', poly),
    ('regressor', model)
])
pipe

In [ ]:
#pipe.fit(X_train, y_train)
#
#model=pipe
#metrics()

#visualizer()

In [ ]:
#valutare di fare la random grid search, che 2025 non sono pochissimissimi
param_grid = {
    #'selection__k': [30, 20, 10],
    'polynomial__degree': [1, 2], #, 3
    'regressor__loss': ['squared_error', 'squared_epsilon_insensitive'], #, 'huber', 'epsilon_insensitive'
    'regressor__penalty': ['l2', 'l1', 'elasticnet', None], 
    'regressor__learning_rate': ['optimal', 'adaptive'], #'constant', 'invscaling',
    'regressor__max_iter': [10000]
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV( #RandomizedSearchCV #GridSearchCV
    estimator=pipe,
    param_grid=param_grid,
    #param_distributions=param_grid,
    #n_iter=10,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring
)

model_grid = grid.fit(X_train, y_train.ravel())
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
##n kerniel dies while plotting the second graph
#visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')

In [ ]:
#########
#########

# MODEL 2 - ENSAMBLE MODELS

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('RANDOM FOREST - PREDICTIONS')

In [ ]:
#########
#########

## HIST GRADIENT BOOSTING

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
dump(model, 'hist_regressor.pkl')
dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('HIST GRADIENT BOOSTING - PREDICTIONS')

In [ ]:
##########
##########

## GRADIENT DESCEND BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('GRADIENT DESCEND BOOSTING - PREDICTIONS')

In [ ]:
#########
#########

## ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = AdaBoostRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')

In [ ]:
#########
#########

## XGBOOST

In [ ]:
from xgboost import XGBRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')